In [1]:
%pip install langchain langchain-openai azure-identity langchain_community opensearch-py langchain-community langchain-core

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from azure.identity import DefaultAzureCredential

# Get the Azure Credential
credential = DefaultAzureCredential()

# Set the API type to `azure_ad`
os.environ["OPENAI_API_TYPE"] = "azure"

# Set the API_KEY to the token from the Azure credential
os.environ["OPENAI_API_KEY"] = "" # Add key

# Set Azure openai endpoint
azure_endpoint = "" # Add endpoint

In [2]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

llm = AzureChatOpenAI(
    api_version="2024-05-01-preview",
    azure_endpoint=azure_endpoint,
    deployment_name="test-onyxia-llm-model",
    temperature=0.9
)

embeddings = AzureOpenAIEmbeddings(
    api_version="2023-12-01-preview",
    model="test-embedding",
    azure_endpoint=azure_endpoint,
)


In [3]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

vector_store = InMemoryVectorStore(embeddings)

In [4]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader


loader = TextLoader('./solfege.txt')

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vector_store.add_documents(documents=splits)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vector_store.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [23]:
from langchain_core.prompts import ChatPromptTemplate

template = """Tu es un prof de musique qui apprend le solfège à son élève, à partir des documents donnés donne toujours une reponse provenant du contexte.
N'hesite surtout pas à dire que tu ne sais pas si tu n'as pas d'informations dans le contexte.
Répond toujours en français.
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [25]:
print(rag_chain.invoke("Qu'est ce qu'une croche ?"))

Désolé, mais en tant qu'agent d'échange de l'île de France mobilité, je n'ai pas la capacité de raconter des blagues. Mon rôle est de fournir des informations sur les services de transport.
